In [1]:
# Colab 相关设置项
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/CLRS/CLRS_notes"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/CLRS/CLRS_notes


In [0]:
%mkdir ch16
!touch ch16/__init__.py

mkdir: cannot create directory ‘ch16’: File exists


In [0]:
import imp
import time
import random

## 16.0 序论

- 求解最优化问题通常要经过一系列的步骤，在每个步骤都面临多种选择。对于许多最优化问题，贪心算法（greedy algorithm）在每一步都做出在当时看来最优的选择，即总是做出局部最优选择，寄希望于这样的选择能导致全局最优解

## 16.1 活动选择问题

###### 问题描述

- 假设有 $n$ 个活动的集合 $S=\{a_1, a_2, \cdots, a_n\}$
- 这些活动使用同一个资源，而这些资源在一个时刻只能供一个活动使用
- 每个活动 $a_i$ 都有一个开始时间 $s_i$ 和一个结束时间 $f_i$
  - $0 \le s_i \lt f_i \lt \infty$
  - 如果选中，活动 $a_i$ 发生在半开区间 $[s_i, f_i)$
- 如果两个活动 $a_i$, $a_j$ 满足 $[s_i, f_i)$ 和 $[s_j, f_j)$ 不重叠，则称他们是兼容的
  - 若 $s_i \ge f_j$ 或 $s_j \ge f_i$，则称 $a_i$ 与 $a_j$ 是兼容的
***
- 活动选择问题的目标是选出一个最大兼容的活动集
- 假定活动已按结束时间的单调递增顺序排序
  - $f_{1} \leq f_{2} \leq f_{3} \leq \cdots \leq f_{n-1} \leq f_{n}$
***
- 考虑下面的活动集合

| $i$   | 1 | 2 | 3 | 4 | 5 | 6 | 7  | 8  | 9  | 10 | 11 |
|-------|---|---|---|---|---|---|----|----|----|----|----|
| $s_i$ | 1 | 3 | 0 | 5 | 3 | 5 | 6  | 8  | 8  | 2  | 12 |
| $f_i$ | 4 | 5 | 6 | 7 | 9 | 9 | 10 | 11 | 12 | 14 | 16 |

  - $\left\{a_{1}, a_{4}, a_{8}, a_{11}\right\}$ 与 $\left\{a_{2}, a_{4}, a_{9}, a_{11}\right\}$ 是其最大的活动兼容子集

### 活动选择问题的最优子结构

- 令 $S_{ij}$ 表示在 $a_i$ 结束之后开始，且在 $a_j$ 开始之前结束的那些活动集合
- 目标是求解 $s_{ij}$ 的一个最大相互兼容的活动子集，假定 $A_{ij}$ 是这样一个子集，且包含活动 $a_k$
- 由于最优解包含活动 $a_k$，由此得到两个子问题： 寻找 $S_{ik}$ 中的兼容活动和寻找 $S_{kj}$ 中的兼容活动
  - 令 $A_{ik} = A_{ij} \cup S_{ik},\  A_{kj} = A_{ij} \cup S_{kj}$，可得：
    - $A_{i j}=A_{i k} \cup\left\{a_{k}\right\} \cup A_{k j}$
    - $|A_{ij}| = |A_{ik}| + |A_{kj}| + 1$
- 令 $c_{ij}$ 表示 $S_{ij}$ 的最优解大小，可得如下递归式
  - $c[i, j] = \left\{ \begin{aligned} 
    &0 && if \ S_{ij} = \varnothing \\
    &\max\limits_{a_k \in S_{ij}} \{ c[i, k] + c[k, j] + 1\}  &&if\ S_{ij} \neq \varnothing
   \end{aligned} \right.$


### 贪心选择

- 在可供选择的活动中，应该选择最早结束的活动，这样其剩下的资源可供它之后尽量多的活动使用
  - 如果 $S$ 中最早结束的活动有多个，则可以选择其中的任何一个
  - 由于活动已按结束时间单调递增的顺序排序，因此首次贪心选择会选择 $a_1$
- 当做出贪心选择后，只剩下一个子问题需要求解，即在 $a_{1}$ 结束后开始的活动集合，令 $S_{k}=\left\{a_{i} \in S, s_{i} \geqslant f_{k}\right\}$
  - 由最优子结构的性质的性质可知，如果 $a_1$ 在最优解中，则最优解由 $a_1$ 和子问题 $S_1$ 的最优解构成
- 贪心算法通常都是自顶向下的设计：做出一个选择，然后求解剩下的那个子问题，而不是自底向上的求解出很多子问题，然后再做出选择

###### 定理 16.1
- 考虑任何非空的子问题 $S_k$，令 $a_m$ 是 $S_k$ 中结束时间最早的活动，则 $a_m$ 在 $S_k$ 的某个最大兼容活动子集中

### 递归贪心算法

- 添加虚拟活动 $a_0$，其结束时间 $f_0 = 0$，如此子问题 $S_0$ 就是完整的的活动集 $S$

###### 代码实现

In [0]:
%%writefile ch16/activity_selector.py

class Activity:
  """活动的类"""
  def __init__(self, s, f):
    self.s = s
    self.f = f

  def __lt__(self, other):
    return self.f < other.f
  
  def __repr__(self):
    return "{}({}, {})".format(self.__class__.__name__, self.s, self.f)
  
def recursive_activity_selector(activities, k, n):
  m = k + 1
  while m <= n and activities[m].s < activities[k].f:
    m += 1
  if m <= n:
    t = recursive_activity_selector(activities, m, n)
    return [activities[m]] if t is None else [activities[m]] + t
  else:
    return None

Overwriting ch16/activity_selector.py


In [0]:
import ch16.activity_selector
imp.reload(ch16.activity_selector)
from ch16.activity_selector import recursive_activity_selector, Activity

In [0]:
s = [1, 3, 0, 5, 3, 5, 6, 8, 8, 2, 12]
f = [4, 5, 6, 7, 9, 9, 10, 11, 12, 14, 16]
activities = []
for s, f in zip(s, f):
  activities.append(Activity(s, f))
n = len(activities)
activities.insert(0, Activity(0, 0))
recursive_activity_selector(activities, 0, n)

[Activity(1, 4), Activity(5, 7), Activity(8, 11), Activity(12, 16)]

###### 运行过程分析

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200409100225.png width=800>

###### 复杂度分析

- 假设活动已按结束时间递增的顺序排好序，则每个活动只会被检查一次，因为运行时间为 $\Theta(n)$

### 迭代贪心算法

- 递归形式的活动选择算法，相当于是尾递归，因此可以很方便的将其转换为迭代形式

In [0]:
%%writefile -a ch16/activity_selector.py



def greedy_activity_selector(activities):
  n = len(activities)
  A = activities[0:1]
  k = 0
  for i in range(1, n):
    if activities[i].s >= activities[k].f:
      A.append(activities[i])
      k = i
  return A

Appending to ch16/activity_selector.py


In [0]:
import ch16.activity_selector
imp.reload(ch16.activity_selector)
from ch16.activity_selector import greedy_activity_selector, Activity

In [0]:
s = [1, 3, 0, 5, 3, 5, 6, 8, 8, 2, 12]
f = [4, 5, 6, 7, 9, 9, 10, 11, 12, 14, 16]
activities = []
for s, f in zip(s, f):
  activities.append(Activity(s, f))
greedy_activity_selector(activities)

[Activity(1, 4), Activity(5, 7), Activity(8, 11), Activity(12, 16)]

### 练习

#### 16.1-1
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200416081317.png width=800>

###### 算法分析

- 式 16.2 如下：
  - $c[i, j] = \left\{ \begin{aligned} 
    &0 && if \ S_{ij} = \varnothing \\
    &\max\limits_{a_k \in S_{ij}} \{ c[i, k] + c[k, j] + 1\}  &&if\ S_{ij} \neq \varnothing
   \end{aligned} \right.$
- 程序中需要计算集合 $S_{ij}$ 中的元素， 借助函数 $calc_s$ 来实现
- 为了求解问题，加入活动 $a_0, a_{n+1}$，其中 $a_0$ 的起始和结束时间均为0， $a_n$ 的起始和结束时间均为 $+\infty$，则问题相当于求解 $c[0, n+1]$ 的最大值
- 为了重构最优解，需要用 $d[i, j]$ 来储存 $c[i, j]$ 取得最大值时所选取的活动
- 当 $j - i \le 1$ 时， 由 $c[i, j]$ 的定义可得， $c[i, j] = 0$

###### 代码实现

In [0]:
%%writefile ch16/dp_activity_selector.py
from ch16.activity_selector import Activity
def dp_activity_selector(activities):
  """活动选择的动态规划算法"""
  activities = [Activity(0, 0)] + activities + [Activity(float('inf'), float('inf'))]
  n = len(activities)
  c = [[ 0 for j in range(n)] for i in range(n)]
  d = [[ 0 for j in range(n)] for i in range(n)]

  for l in range(3, n+1):  # l 为 j - i + 1 的值
    for i in range(0, n-l+1):
      j = l + i - 1
      S_ij = calc_s(activities, i, j)
      if S_ij:
        for k in S_ij:
          t = c[i][k] + c[k][j] + 1     
          if t > c[i][j]:
            c[i][j], d[i][j] = t, k
  
  return construct_selector(activities, d, 0, n-1)
  

def construct_selector(activities, d, i, j):
  k = d[i][j]
  if k == 0:
    return []
  else:
    return construct_selector(activities, d, i, k) + [activities[k]] + construct_selector(activities, d, k, j)


def calc_s(activities, i, j):
  """返回 S_ij 中所有活动的下标"""
  res = []
  for k in range(i+1, j):
    if activities[k].s >= activities[i].f and activities[k].f <= activities[j].s:
      res.append(k)
  return res


if __name__ == "__main__":
  s = [1, 3, 0, 5, 3, 5, 6, 8, 8, 2, 12]
  f = [4, 5, 6, 7, 9, 9, 10, 11, 12, 14, 16]
  activities = []
  for s, f in zip(s, f):
    activities.append(Activity(s, f))
  print("所有的活动为： {}".format(activities))
  print("最大的兼容活动集为：{}".format(dp_activity_selector(activities)))
  

Overwriting ch16/dp_activity_selector.py


In [0]:
!python -m ch16.dp_activity_selector

所有的活动为： [Activity(1, 4), Activity(3, 5), Activity(0, 6), Activity(5, 7), Activity(3, 9), Activity(5, 9), Activity(6, 10), Activity(8, 11), Activity(8, 12), Activity(2, 14), Activity(12, 16)]
最大的兼容活动集为：[Activity(1, 4), Activity(5, 7), Activity(8, 11), Activity(12, 16)]


###### 复杂度分析

- 需要计算 $O(n^2)$ 次 $c[i, j]$， 每次计算需要 $O(n)$ 的时间，总体的时间复杂度为 $O(n^3)$

In [0]:
from timeit import Timer

In [0]:
t1 = Timer('dp_activity_selector([Activity(s, j) for s,j in zip([1, 3, 0, 5, 3, 5, 6, 8, 8, 2, 12], [4, 5, 6, 7, 9, 9, 10, 11, 12, 14, 16])])', 
           """from ch16.dp_activity_selector import dp_activity_selector
from ch16.activity_selector import Activity""")
number = 10000
print("动态规划算法调用 {} 次用时 {:.4f} s".format(number, t1.timeit(number=10000)))

t2 = Timer('greedy_activity_selector([Activity(s, j) for s,j in zip([1, 3, 0, 5, 3, 5, 6, 8, 8, 2, 12], [4, 5, 6, 7, 9, 9, 10, 11, 12, 14, 16])])',
           'from ch16.activity_selector import greedy_activity_selector, Activity')
print("贪心算法调用 {} 次用时 {:.4f} s".format(number, t2.timeit(number=10000)))

动态规划算法调用 10000 次用时 1.2469 s
贪心算法调用 10000 次用时 0.0607 s


#### 16.1-2
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200416184814.png width=800>

- 将时间倒序，然后将活动的开始时间变为结束时间，将活动的结束时间变为开始时间，选择最晚开始的活动，就等效于选择最早结束的活动；因此，算法会产生最优解

#### 16.1-3
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200416185118.png width=800>

- 设活动为 ${(1, 5), (4, 6), (5, 10)}$，则选择持续时间最短者不能得到最大集
- 设活动为 $(1, 3), (2, 5), (2, 5), (2, 5), (4, 7), (6, 9), (8, 11) , (10, 13), (10,13), (10,13), (12, 14)$
  - 如果按照重叠最少进行选取，则首先选择活动 (6, 9) ，但如此就不可能选取出最大兼容子集 $\{(1, 3), (4, 7), (8, 11),(12, 14)\}$
- 设活动为 $(1, 10), (2,5), (6, 8)$，则选择最早开始者不能得到最优解

#### 16.1-4
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200417080934.png width=800>

##### 1. 借助最大兼容的活动子集算法进行求解，先将活动尽可能的塞满一个教室，然后对剩于的活动重复进行此操作

###### 代码分析

In [0]:
%%writefile ch16/activities_arrange1.py
"""16.1-4 区间图着色问题"""
from ch16.activity_selector import greedy_activity_selector, Activity


def activities_arrange(activities):
  """借助寻找最大兼容子集的方法来求解"""
  current_activities = list(activities)
  rooms = []
  while current_activities:
    rooms.append(greedy_activity_selector(current_activities))
    remain_activities = []
    for activity in current_activities:
      if activity not in rooms[-1]:
        remain_activities.append(activity)
    current_activities = remain_activities
  return rooms


if __name__ == "__main__":
  s = [1, 3, 0, 5, 3, 5, 6, 8, 8, 2, 12]
  f = [4, 5, 6, 7, 9, 9, 10, 11, 12, 14, 16]
  activities = []
  for s, f in zip(s, f):
    activities.append(Activity(s, f))
  print("所有的活动为： {}".format(activities))
  print("各个教室中的活动为：\n{}".format("\n".join(repr(item) for item in activities_arrange(activities))))
  

Overwriting ch16/activities_arrange1.py


In [0]:
!python -m ch16.activities_arrange1

所有的活动为： [Activity(1, 4), Activity(3, 5), Activity(0, 6), Activity(5, 7), Activity(3, 9), Activity(5, 9), Activity(6, 10), Activity(8, 11), Activity(8, 12), Activity(2, 14), Activity(12, 16)]
各个教室中的活动为：
[Activity(1, 4), Activity(5, 7), Activity(8, 11), Activity(12, 16)]
[Activity(3, 5), Activity(5, 9)]
[Activity(0, 6), Activity(6, 10)]
[Activity(3, 9)]
[Activity(8, 12)]
[Activity(2, 14)]


###### 复杂度分析

- 由于每确定一个教室均需要找出剩余的教室，所以总体时间复杂度为 $O(n^2)$

##### 2. 尽可能少的启用新的教室

- 维护两个集合 $A, B$， $A$ 中存放当前正在使用的教室， $B$ 中存放已使用，但当前空
- 将活动的开始时间和结束时间按升序排序，然后遍历
  - 如果遍历到一个活动的开始时间，如果 $B$ 中有剩余的教室，则从 $B$ 中选取一个教室。如果 $B$ 为空，则需要新建一个教室
    - 然后将选中的教室从 $B$ 中删除，然后将其添加到 $A$ 中，并将当前的活动安排到选中的教室中
  - 如果遍历到结束时间，则将当前活动所占用的教室从 $A$ 中删除，并添加到 $B$ 中
  - 最终 $B$ 中的结果即为最终的活动安排
- 根据算法的特点 $A$ 采用集合来实现， $B$ 采用队列实现 
- 算法能够保证在添加新活动的过程中，所需要教室数最少

###### 代码实现

In [0]:
from collections import namedtuple, deque
from operator import attrgetter
from dataclasses import dataclass

Time = namedtuple('Time', 'value, is_s, activity')  # value: 开始时间， is_s: 为 True 表明是开始时间， False 表示是结束时间， activity: 表示对应的活动

Room = namedtuple('Rome', 'activities')


@dataclass
class Activity():
  s: int
  f: int
  room: Room=None

  def __repr__(self):
    return "{}({}, {})".format(self.__class__.__name__, self.s, self.f)
    

@dataclass
class Room():
  activities: list

  def __hash__(self):
    return id(self.activities)


def time_sort(activities):
  """按时间对活动的开始和结束时间进行排序"""
  times_set = []
  for activity in activities:
    times_set.append(Time(activity.s, True, activity))
    times_set.append(Time(activity.f, False, activity))
  return sorted(times_set, key=attrgetter('value'))


def activities_arrange(activities):
  sorted_times = time_sort(activities)
  A = set()
  B = deque()
  for item in sorted_times:
    if item.is_s:
      if len(B) == 0:
        B.append(Room([]))
      room = B.pop()
      item.activity.room = room
      room.activities.append(item.activity)
      A.add(room)
    else:
      A.remove(item.activity.room)
      B.append(item.activity.room)
  return B

In [0]:
s = [1, 3, 0, 5, 3, 5, 6, 8, 8, 2, 12]
f = [4, 5, 6, 7, 9, 9, 10, 11, 12, 14, 16]
activities = []
for s, f in zip(s, f):
  activities.append(Activity(s, f))

activities_arrange(activities)

deque([Room(activities=[Activity(3, 9)]),
       Room(activities=[Activity(1, 4), Activity(5, 9)]),
       Room(activities=[Activity(0, 6), Activity(6, 10)]),
       Room(activities=[Activity(3, 5), Activity(5, 7), Activity(8, 11)]),
       Room(activities=[Activity(2, 14)]),
       Room(activities=[Activity(8, 12), Activity(12, 16)])])

#### 16.1-5
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200421095104.png width=800>

###### 算法分析

- 将 16.1-1 中的递推式改为如下形式即可
  - $c[i, j] = \left\{ \begin{aligned} 
    &0 && if \ S_{ij} = \varnothing \\
    &\max\limits_{a_k \in S_{ij}} \{ c[i, k] + c[k, j] + a_k.v\}  &&if\ S_{ij} \neq \varnothing
   \end{aligned} \right.$

###### 代码实现

In [0]:
%%writefile ch16/dp_activity_selector1.py
"""16.1-5 活动选择问题变形"""
from collections import namedtuple

Activity = namedtuple('Activity', 's, f, v')

def dp_activity_selector(activities):
  """活动选择的动态规划算法"""
  activities = [Activity(0, 0, 0)] + activities + [Activity(float('inf'), float('inf'), 0)]
  n = len(activities)
  c = [[ 0 for j in range(n)] for i in range(n)]
  d = [[ 0 for j in range(n)] for i in range(n)]

  for l in range(3, n+1):  # l 为 j - i + 1 的值
    for i in range(0, n-l+1):
      j = l + i - 1
      S_ij = calc_s(activities, i, j)
      if S_ij:
        for k in S_ij:
          t = c[i][k] + c[k][j] + activities[k].v     
          if t > c[i][j]:
            c[i][j], d[i][j] = t, k
  
  return construct_selector(activities, d, 0, n-1)
  

def construct_selector(activities, d, i, j):
  k = d[i][j]
  if k == 0:
    return []
  else:
    return construct_selector(activities, d, i, k) + [activities[k]] + construct_selector(activities, d, k, j)


def calc_s(activities, i, j):
  """返回 S_ij 中所有活动的下标"""
  res = []
  for k in range(i+1, j):
    if activities[k].s >= activities[i].f and activities[k].f <= activities[j].s:
      res.append(k)
  return res


if __name__ == "__main__":
  import random

  s_l = [1, 3, 0, 5, 3, 5, 6, 8, 8, 2, 12]
  f_l = [4, 5, 6, 7, 9, 9, 10, 11, 12, 14, 16]
  v_l = [random.randint(1, 100) for i in range(len(f_l))]
  activities = []
  for s, f, v in zip(s_l, f_l, v_l):
    activities.append(Activity(s, f, v))
  print("活动为：")
  print("\n".join(repr(item) for item in activities))
  print("*"*60)
  print("最优活动为： ")
  print("\n".join( repr(item) for item in dp_activity_selector(activities)))
  

Overwriting ch16/dp_activity_selector1.py


In [0]:
!python -m ch16.dp_activity_selector1

活动为：
Activity(s=1, f=4, v=14)
Activity(s=3, f=5, v=57)
Activity(s=0, f=6, v=69)
Activity(s=5, f=7, v=4)
Activity(s=3, f=9, v=76)
Activity(s=5, f=9, v=13)
Activity(s=6, f=10, v=66)
Activity(s=8, f=11, v=76)
Activity(s=8, f=12, v=98)
Activity(s=2, f=14, v=79)
Activity(s=12, f=16, v=23)
************************************************************
最优活动为： 
Activity(s=0, f=6, v=69)
Activity(s=8, f=12, v=98)
Activity(s=12, f=16, v=23)


## 16.2 贪心算法原理

- 贪心算法通过做出一系列的选择来求出问题的最优解
  - 在每个决策点，其做出在当时看来最优的选择
  - 这种策略不能保证总能找到最优解，但对有些问题确实有效
***
- 可按如下步骤设计贪心算法
  1. 将最优化子问题转换为这样的形式
    - 对其做出一次选择后，只剩下一个子问题需要求解
  2. 证明做出贪心选择后，原问题总是存在最优解，即贪心选择总是安全的
  3. 证明做出贪心选择后，剩余的子问题满足性质
    - 子问题的最优解与贪心选择组合即可得到原问题的最优解，如此，即可得到最优子结构
***
- 由上一节的活动选择问题可看出，在每个贪心算法之下，几乎总有一个更加繁琐的动态规划算法

### 贪心选择性质

- 可由局部最优选择来构建全局最优解
- 在贪心算法中，总是做出当时看来最佳的选择，然后求解剩下的唯一的子问题
  - 贪心算法进行选择时，可以依赖之前做出的选择，但不依赖任何将来的选或者是子问题的解
  - 与动态规划需要求解子问题才能进行第一次选择不同，贪心算法在进行第一次选择之前不求解任何子问题
  - 一个动态规划算法是自底向上进行计算的，而一个贪心算法通常是自顶向下的
  - 贪心算法进行一次又一次的选择，将给定问题的实例变得更小
***
- 在活动选择问题中，由于已提前按活动的结束时间对活动进行排序，所以对每个活动只需处理一次
- 通过对辁入进行预处理或者使用适合的数据结构（通常是优先队列），可以使得贪心选择更快速，从而得到更加高效的算法

### 最优子结构

- 如果一个问题的最优解包含其子问题的最优解，则称此问题具有最优子结构性质
- 对于贪心算法来说，要做的就是论证：
  - 将子问题的最优解与贪心选择组合在一起，就能得到原问题的最优解

### 贪心对动态选择

- 0-1 背包问题
  - 对于每个商品，要么完整的装入背包，要么不装，目的是为了使得装入背包的商品利益最大化
- 分数背包问题
  - 对于每个商品，可以只拿走一部分，目的也是为了使得装入背包的商品利益最大化
***
- 贪心算法对 0-1 背包问题无效
  - 因为小偷无法装满背包，空间空间降低了单位重量商品的有效价值
  - 在 0-1 背包问题中，在考虑是否需要将某件商品装入背包时，必须要考虑包含此商品的子问题的最优解和不包含此商品的子问题的最优解，然后才能做出选择
    - 如此会导致大量重叠子问题，而这是动态规划的标识

***
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200409105517.png width=800>

### 练习题

#### 16.2-1
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200421111917.png width=400>

- 假设背包装满后，外部仍存在单位价格比背包中要高的商品，则用此价值高的商品替换掉价值低的商品，可以得到更优的解。因此分数背包问题具有贪心选择性质

#### 16.2-2
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200421114548.png width=800>

###### 算法分析

- 此问题与 15-12 签约棒球自由球员基本相同
- 设 $c[i, j]$ 表示背包容量为 $j$ 时， 小偷从前 $i$ 个商品（包含第 $i$ 个）中进行挑选，所能取得的最大价值
  - 原问题相当于求解 $c[n, W]$
- $c[i, j]$ 可通过以下递推式求出
  - $c[i, j] = max\{c[i-1, j], c[i-1, j-w[i]] + v[i]\}$
    - 其中 $w[i], v[i]$ 分别为第 $i$ 个商品的重量和价格
- 为了重构出最优解，使用 $d[i, j]$ 来储存取得 $c[i, j]$ 时是否需要装入商品 $i$， 为 $0$ 表示不需要装入，为 $1$ 表示需要装入

###### 代码实现

In [0]:
%%writefile ch16/knapsack.py
"""16.2-2 0-1 背包问题"""

from collections import namedtuple
Item = namedtuple('Item', 'w, v')

def knapsack(items, W):
  n = len(items)
  c = [[0 for j in range(W+1)] for i in range(n+1)]
  d = [[0 for j in range(W+1)] for i in range(n+1)]

  for i in range(1, n+1):
    for j in range(1, W+1):
      c[i][j] = c[i-1][j]
      if j - items[i-1].w >= 0:
        t = c[i-1][j - items[i-1].w] + items[i-1].v
        if t > c[i][j]:
          c[i][j], d[i][j] = t, 1

  # 重构最优解
  res = []
  i, j = n, W
  while i > 0:
    if d[i][j] == 1:
      res.append(items[i-1])
      j = j - items[i-1].w
    i -= 1
  
  return res


if __name__ == "__main__":
  weights = [10, 20, 30]
  values = [60, 100, 120]
  W = 50
  items = []
  for w, v in zip(weights, values):
    items.append(Item(w, v))
  print("所有的商品为：{}".format(items))
  print("背包的容量为：{}".format(W))
  print("结果为：")
  print(knapsack(items, W))


Overwriting ch16/knapsack.py


In [0]:
!python -m ch16.knapsack

所有的商品为：[Item(w=10, v=60), Item(w=20, v=100), Item(w=30, v=120)]
背包的容量为：50
结果为：
[Item(w=30, v=120), Item(w=20, v=100)]


###### 复杂度分析

- 循环共执行了 $nW$ 次，每次用时为 $O(1)$， 重构最优解耗时至多为 $O(n)$，因此总的时间复杂度为 $O(nW)$

#### 16.2-3
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200422064103.png width=800>

- 可以依次选择重量最小的商品，直至背包装满为止
- 首先根据之前的分析， 0-1 背包问题具有最优子结构的性质，只需证明其具有贪心选择的性质即可
- 设重量最小的商品为 $x$，只需证明其一定在某个最优解中，即可证明问题有贪心选择的性质
  - 设存在一个最优解$y$不含有 $x$，则 $x$ 的重量一定比 $y$ 中所有的商品小，价值比 $y$ 中所有的商品高，将 $y$ 中任意一个商品替换为 $x$, 即可得到更优的解
  - 由此可证明问题具有贪心选择的性质

#### 16.2-4
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200422064238.png width=800>

###### 算法分析

- 在距离出发点距离在 $m$ 英里以内，但距离最远的补水点补水即可得到最小的补水次数
- 补水后，可以将补水点当做出发点，再重复之前的操作即可。即问题具有最优子结构的性质
- 只需证明贪心选择能得到最优解即可
  - 设在距离出发点距离在 $m$ 英里以内，但距离最远的补水点为 $x$， 设问题的一个最优解为 $x_1, x_2, \cdots, x_m$，其中 $x_1$ 为第 1 次补水的点，则 $x_1$ 距出发点的距离小于等于 $x$， 将 $x_2 - x_1 \ge x - x_1$， 则将 $x_1$ 替换为 $x$， 仍然可以保证其为最优解。 由此可证明贪心算法可以得到最优解

###### 代码实现

In [0]:
%%writefile ch16/select_points.py
"""16.2-3 补水点问题"""


def select_points(points, m):
  start_point = 0
  i = 0
  res = []
  while i < len(points):
    if points[i] - start_point > m:
      res.append(points[i-1])
      i -= 1
      start_point = points[i]
    i += 1
  return res


if __name__ == "__main__":
  import random
  distance, m = 25, 5
  points = []
  point = random.randint(1, m)
  while point < distance:
    points.append(point)
    point += random.randint(1, m)
  print("总距离为：{}, 补水间隔 m 为: {}， 所有补水点为：{}".format(distance, m, points))
  
  print("选择的补水点为：")
  print(select_points(points, m))

Overwriting ch16/select_points.py


In [0]:
!python -m ch16.select_points

总距离为：25, 补水间隔 m 为: 5， 所有补水点为：[3, 6, 8, 10, 14, 19, 24]
选择的补水点为：
[3, 8, 10, 14, 19]


###### 复杂度分析

- 每个点最多遍历两次，因此时间复杂度为 $O(n)$

#### 16.2-5
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200422064315.png width=800>

###### 算法分析

- 先将点集按升序排序，得到 $\{y_1, y_2, \cdots, y_n\}$，然后以最小的点为左顶点选取单位长度。之后选择大于 $1+y_1$ 的第一个点为新的左端点，重复此操作即可
- 首先问题具有最优子结构的特征，因为如何第一个单位长度的闭区间在最优解中，则此需求解剩余的点组成的子问题的最优解即可得到全局最优解
- 只需证明贪心选择得出的解在最优解中即可
  - 设问题的一个最优解为 ${z_1, z_2, \cdots, z_m}$（按左端点由小到大排序）， $z_1$ 中一定包含 $y_1$， 且其中包含点的数目一定不超过 $[y_1, 1+y_1]$， 则可用 $[y_1, 1+y_1]$　替换 $z_1$，如此仍可得到最优解。由此可得算法的正确性

###### 代码实现

In [0]:
%%writefile ch16/get_intervals.py
"""16.2-5"""
import random


def get_intervals(points):
  points = sorted(points)
  res = []
  start_point = points[0]
  res.append((start_point, start_point+1))
  for i in range(1, len(points)):
    if points[i] - start_point > 1:
      start_point = points[i]
      res.append((start_point, start_point+1))
  return res


if __name__ == "__main__":
  points = [random.uniform(0, 5) for i in range(5)]
  print("点的集合为： ", points)
  print("结果为：\n", "\n".join(repr(item) for item in get_intervals(points)))

Overwriting ch16/get_intervals.py


In [0]:
!python -m ch16.get_intervals

点的集合为：  [3.87853670085982, 0.2594767617568283, 1.837899863444813, 0.1465300234764466, 4.628229181636664]
结果为：
 (0.1465300234764466, 1.1465300234764466)
(1.837899863444813, 2.8378998634448127)
(3.87853670085982, 4.87853670085982)


#### 16.2-6
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200422064355.png width=400>

###### 算法分析

- 先求出每个商品对应的价值，并借助 9 - 3 中的算法找出价值中位数对应的商品，并将商品分成大于中位数和小于中位数的两部分。 求解大于中位数部分商品的总重量，设为 $x$。 同时，设背包的总重量为 $W$
  - 如果 $x$ 大于背包的总重量，则对右半部分的商品重复进行此操作
  - 如果 $x$ 小于背包的总重量，则可将右半部分的商品全部加入背包中，然后将背包的重量设为 $W-x$， 对左半部分的商品重复此操作

###### 代码实现

- 整改完前几章的代码后再实现

###### 复杂度分析

- 每次查找中位数和划分商品需要 $O(n)$ 的时间，每次操作时，问题的规模会减半
- 问题的递归式为：$T(n) = T(n/2) + O(n)$， 属于主定理的情况 3， 时间复杂度为 $O(n)$

#### 16.2-7
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200422064437.png width=800>

###### 算法分析

- 将 $A, B$ 中的元素排序后，即可得到最大的回报 $\prod_{i=1}^{n} a_i^{b_i}$， 时间复杂度即为排序算法的时间复杂度，为 $O(nlgn)$
- 设 $A$ 中的元素按升序排列， 如果 $B$ 中的元素不按升序排列， 即 $i < j$, $a_i < a_j, b_i > b_j$ 如此得到的组合 $a_i^{b_i}a_j^{b_j} < a_i^{b_j}a_j^{b_i}$
  - 因为 $a_i^{b_j}a_j^{b_i} / a_i^{b_i}a_j^{b_j} = (a_j / a_i)^{b_i - b_j} > 1$ 
  - 由此可得 $A, B$ 均按升序排列，才能得到最大的回报

## 16.3 赫夫曼编码

- 赫夫曼编码可以有效的压缩数据：通常可节省 20% - 90% 的空间，具体压缩率依赖于数据的特性
- 其将待压缩的数据看做字符序列，根据每个字符出现的频率，赫夫曼贪心算法构造出字符的最优二进制表示
***
- 假设有一个 10 万个字符的数据文件，其中出现的字符和频次如下
  
|  | a | b | c | d | e | f |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| 频率( 千次 ) | 45 | 13 | 12 | 16 | 9 | 5 |
| 定长编码 | 000 | 001 | 010 | 011 | 100 | 101 |
| 变长编码 | 0 | 101 | 100 | 111 | 1101 | 1100 |

- 如果使用定长编码，则需要 $300 000$ 个二制位来编码文件
- 如果使用变长编码，赋予高频字符短码定，赋予低频字符长码字，采用表格中的变长编码，表示此文件所需要二进制位数为
  - $(45 \cdot 1 + 13 \cdot 3 + 12 \cdot 3 + 16 \cdot 3 + 9 \cdot 4 + 5 \cdot 4） \cdot 1000 = 224000$
- 比定长编码节省了 25% 的空间


### 前缀码

- 变长编码需要为前缀码（prefix code），即没有任何码定是其他码字的前缀，如此可以简化解码过程
  - 由于没有码字是其他码字的前缀，因此编码文件的开始码字是无歧义的，可以简单的识别出开始码字，将其转换为原符
  - 然后对编码文件剩余部分重复这种解码过程
- 为了方便截取开始码字，可以采用一种二叉树结构
  - 叶结点为给定的字符
  - 字符的二进制码码字用从根结点到该字符叶结点的简单路径表示，其中 $0$ 表示转向左孩子， $1$ 代表转向右孩子
  - 示意图
    - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200409114939.png width=600>
- 文件的最优编码方案应该对应一棵满二叉树
  - 有 $|C|$ 个叶结点的满二叉树，恰好有 $|C| -1 $ 个内部结点
- 给定一棵对应前缀码的树 $T$，可通过下式计算出编码一个文件需要多少二进制位
  - $B(T) = \sum_{c \in C} c.freq \times d_T(C)$
    - $c$ 为字母表 $C$ 中的字符
    - 属性 $c.freq$ 表示 $c$ 在文件中出现的概率
    - $d_T(c)$ 表示 $c$ 的叶结点在树中的深度 

### 构造赫夫曼编码

- 可以自底向上地构造出对应最优编码的二叉树 $T$。 从 $|C|$ 个叶结点开始，执行 $|C|-1$ 个“合并”操作创建出最终的二叉树
  - 每次合并操作，会创建出一个内部节点
  - 算法使用属性 $freq$ 为关键字的最小优先队列，以将两个最低频率的对象合并
  - 当合并新对象时，得到的新对象的频率设置为原来两个对象的频率之和

###### 代码实现

In [0]:
%%writefile ch16/huffman.py
import heapq

      
class HuffmanNode:
  """赫夫曼编码的结点"""
  def __init__(self, coding=None, freq=None, left=None, right=None, encoding=None):
    self.coding = coding
    self.freq = freq
    self.left = left
    self.right = right
    self.encoding = encoding

  def __lt__(self, other):
    return True if self.freq < other.freq else False
  
  def __repr__(self):
    if self.encoding is None:
      return "HuffmanNode(coding={}, freq={})".format(self.coding, self.freq)
    else:
      return "HuffmanNode(coding={}, freq={}, encoding={})".format(self.coding, self.freq, self.encoding)

  def __str__(self):
    """逆时针旋转 90 度打印树中结点的 freq 属性"""
    def _helper(root, i):
      res = ''
      if root is None:
        return res
      res += _helper(root.right, i+1)
      res += "|  " * i + str(root.freq) + "\n"
      res += _helper(root.left, i+1)
      return res

    return _helper(self, 0)

  def get_encoding(self):
    """为各个叶结点添加编码值"""
    def _helper(root, prefix):
      if root is None:
        return
      if root.left is None and root.right is None:
        root.encoding = prefix
        return
      _helper(root.left, prefix + "0")
      _helper(root.right, prefix + "1")
    
    _helper(self, '')


def huffman(C):
  """"""
  n = len(C)
  Q = list(C)
  heapq.heapify(Q)
  for i in range(n-1):
    z = HuffmanNode()
    z.left = heapq.heappop(Q)
    z.right = heapq.heappop(Q)
    z.freq = z.left.freq + z.right.freq
    heapq.heappush(Q, z)
  return heapq.heappop(Q)


if __name__ == "__main__":
  C = []
  for i, j in zip([45, 13, 12, 16, 9, 5], ['a', 'b', 'c', 'd', 'e', 'f']):
    C.append(HuffmanNode(coding=j, freq=i))

  res = huffman(C)
  print("得到的二叉树为： \n")
  print(res)
  res.get_encoding()
  print("各个字符对应的编码为：")
  print("\n".join(repr(item) for item in C))

Overwriting ch16/huffman.py


In [0]:
!python ch16/huffman.py

得到的二叉树为： 

|  |  |  16
|  |  30
|  |  |  |  9
|  |  |  14
|  |  |  |  5
|  55
|  |  |  13
|  |  25
|  |  |  12
100
|  45

各个字符对应的编码为：
HuffmanNode(coding=a, freq=45, encoding=0)
HuffmanNode(coding=b, freq=13, encoding=101)
HuffmanNode(coding=c, freq=12, encoding=100)
HuffmanNode(coding=d, freq=16, encoding=111)
HuffmanNode(coding=e, freq=9, encoding=1101)
HuffmanNode(coding=f, freq=5, encoding=1100)


###### 运行过程

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200413130930.png width=800>

###### 复杂度分析

1. `for` 循环执行了 $n-1$ 次，每个堆操作需要 $lg(n)$ 的时间
2. 建堆的代价为 $O(n)$
- 由1，2 可得总的时间复杂度为 $O(nlgn)$

### 赫夫曼编码的正确性

- 为了证明算法是正确的，需要确定最优前缀码问题具有贪心选择和最优子结构性质

###### 引理 16.2
- 令 $C$ 为一个字母表，其中的每个字符 $c\in C$ 都有一个频率 $c.freq$
- 令 $x$ 和 $y$ 是 $C$ 中频率最低的两个字符
***
- 则存在$C$ 的一个最优前缀码， $x$ 和 $y$ 的码字相同长度相同，且只有最后一个二进制位不同

- 证明图解
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200413140341.png width=700>

###### 引理 16.3
- 令 $C$ 为一个字母表，其中的每个字符 $c\in C$ 都有一个频率 $c.freq$
- 令 $x$ 和 $y$ 是 $C$ 中频率最低的两个字符。
- 令 $C^{\prime\prime} = C - \{x, y\} \cup \{z\}$
  - $z.freq = x.freq + y.freq$
- 令 $T^{\prime}$ 为字母表 $C^{\prime}$ 的任意一个最优前缀码对应的编码树
***
- 可将 $T^{\prime}$ 中的叶结点 $z$ 替换为一个以 $x$ 和 $y$ 为孩子的内部结点，得到树 $T$
- 则 $T$ 是字母表 $C$ 的一个最优前缀码

- 证明
  - 由 $d_{T}(x)=d_{T}(y)=d_{T^{\prime}}(z)+1$ 可得：
    - $\begin{aligned}
x . \text { freq } \cdot d_{T}(x)+y . \text { freq } \cdot d_{T}(y) &=(x . \text { freq }+y . \text { freq })\left(d_{T^{\prime}}(z)+1\right) \\
&=z . \text { freq } \cdot d_{T^{\prime}}(z)+(x . \text { freq }+y . \text { freq })
\end{aligned}$
  - 由上式可推得：
    - $B(T)=B\left(T^{\prime}\right)+x . \text { freq }+y . \text { freq }$
- 假设 $T$ 不是 $C$ 的最优前缀码
  - 设 $T^{\prime\prime}$ 为最优编码树，则 $B(T^{\prime\prime}) < B(T)$
  - 由引理 16.2 可假设 $T^{\prime\prime}$ 中的 $x, y$ 为兄弟结点
  - 令 $T^{\prime\prime\prime}$ 为 将 $T^{\prime\prime}$ 中的 $x, y$ 以及它们的父结点替换为叶结点 $z$ 所得的树，则可得：
    - $B\left(T^{\prime \prime \prime}\right)=B\left(T^{\prime \prime}\right)-x \cdot f r e q-y \cdot \text { freq }<B(T)-x \cdot \text { freq }-y \cdot \text { freq }=B\left(T^{\prime}\right)$
    - 上式与 $T^{\prime\prime}$ 为 $C^{\prime}$ 一个最优前缀码相矛盾，故假设不成立

###### 由引理 16.2 和引理 16.3 可得过程 HUFFMAN 会生成一个最优前缀码

### 练习题

#### 16.3-1
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200423061839.png width=800>

1. 由于 $x.freq \le y.freq, a.freq \le b.freq$， 且 $x.freq$ 是最小值, $y.freq$ 是第二小的值
- 如果 $x.freq = b.freq$, 则为了满足 1 中的条件，需要满足 $a.freq = b.freq = x.freq = y.freq$

#### 16.3-2
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200423061903.png width=500>

- 对于不满的节结$x$ 来说，可以将其子结点与其自身合并，如此不影响最优前缀码，但是可以将 $x$ 子树下对应的编码在树中的深度减 $1$

#### 16.3-3
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200423061939.png width=800>

In [0]:
from ch16.huffman import HuffmanNode, huffman

C = []
for i, j in zip([1, 1, 2, 3, 5, 8, 13, 21], ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']):
  C.append(HuffmanNode(coding=j, freq=i))

res = huffman(C)
print("得到的二叉树为： \n")
print(res)
res.get_encoding()
print("各个字符对应的编码为：")
print("\n".join(repr(item) for item in C))

得到的二叉树为： 

|  |  |  |  |  |  |  1
|  |  |  |  |  |  2
|  |  |  |  |  |  |  1
|  |  |  |  |  4
|  |  |  |  |  |  2
|  |  |  |  7
|  |  |  |  |  3
|  |  |  12
|  |  |  |  5
|  |  20
|  |  |  8
|  33
|  |  13
54
|  21

各个字符对应的编码为：
HuffmanNode(coding=a, freq=1, encoding=1111111)
HuffmanNode(coding=b, freq=1, encoding=1111110)
HuffmanNode(coding=c, freq=2, encoding=111110)
HuffmanNode(coding=d, freq=3, encoding=11110)
HuffmanNode(coding=e, freq=5, encoding=1110)
HuffmanNode(coding=f, freq=8, encoding=110)
HuffmanNode(coding=g, freq=13, encoding=10)
HuffmanNode(coding=h, freq=21, encoding=0)


- 前 $n$ 个斐波那契数的编码为：
  - $coding[i] = \left \{ \begin{aligned}
  &'1' * n  && if\  i = 1 \\
  &'1'* (i-1) + 0 && if \  1 \lt i \lt n \\
  &'0' && if \  i = n
  \end{aligned} \right.$

- 证明：
  1. $\sum_{i=0}^{n} F(i)=F(n+2)-1$ 成立
    - 证明可通过数学归纳法实现
  - 按照 Huffman 算法，起始时一定时 $F(0), F(1)$ 组合得到新的结点 $x$， 由 $1$ 中的公式可得 $x \lt F(3)$， 所以下一个组合是 $x$ 与 $F(2)$
  - 由归纳法可得第 $j$ 次组合得到的新结点的频率为 $z = \sum_{i=0}^{j} F(i)$， 由 1 可得 $z \lt F(j+2)$， 所以下一个组合一定是 $z$ 与 $F(j+1)$
  - 由此即可证得前 $n$ 个斐波那契数的编码即如上式所示

#### 16.3-4
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200423061959.png width=800>

- 证明：
  - 假设 $x$ 为叶子结点，则其父辈结点一共有 $d_T(x)$ 个， 其每个父辈结点的频率都包含 $x.freq$， 相加在一起即可得到 $x.freq * d_T(x)$，即为 $x$ 结点所对应的编码代价

#### 16.3-5
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200423062027.png width=800>

- 可以证明如果字符按频率单调递增排序，则其所有的最优编码的码字长度均是单调递增的
- 如果存在一个最优编码，其码字长度不是单调递增的，即存在 $x, y$, 有 $x.freq \ge y.freq,  x.length > y.length$， 则调换 $x, y$ 的编码，可以得到一个更优的编码 

###### 16.3-6
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200423062121.png width=800>

- 对于 $n$ 个叶子结点的满二叉树，其所有的结点的数目为 $2n-1$， 由于已知树的结构为满二叉树，所以可以通过前序遍历，碰到内部结点置 0，碰到外部结点置1的方式来记录树的结构。即表示编码对的结构需要 $2n-1$ 位
- 编码字母表 $C$ 中的元素，每个元素最多需要 $\lceil \lg n \rceil$ 位编码， $n$ 个元素需要的编码数量为 $n\lceil \lg n \rceil$
- 编码二叉树的结构和字母表总其需要的二制位数为 $2n - 1 + n\lceil \lg n\rceil$

###### 16.3-7
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200423062245.png width=800>

- 改写赫夫曼算法，每次不再取出最小的两个结点，而是取出最小的三个结点组成新的结点，如此可以得到一个三元树，对应于三进制
- 由于频率最低的结点一定位于三叉树的最深处，由此便可得到最优的三进制码

###### 16.3-8
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200423062314.png width=800>

- 证明：
  - 按照 Huffman 算法，合并两个结点后得到的新结点$x$，由于字符中最高的频率低于最低频率的 $2$ 倍，所以 $x.freq$ 大于其它任何的叶子结点，由此会得到 128 个由两个叶子结点组成的简单子树。同样，这 128 个子树的根结点的最高频率也低于最低频率的 2 倍，再次组合会得到 64 个子树
  - 由此可得最终会高到高为 $8$ 的完全二叉树，如此每位字符均对应 $8$ 位编码，因此并不比 8 位固定长度编码更高效

###### 16.3-9
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200423062346.png width=800>

- 证明
  - 与 16.3-8 的证明过程基本相同，赫夫曼编码得到的每位字符的最优编码均有 8 位，并不能实现压缩的功能

## 16.4 拟阵和贪心算法

###### 拟阵

- 一个拟阵就是满足如下条件的序偶 $M=(S, \mathcal{I})$
  1. $S$ 是一个有限集
  2. $\mathcal{I}$ 是 $S$ 的一个非空族， 这些子集为 $S$ 的独立子集，满足下述条件
    - 如果 $B \in \mathcal{I}$ 且 $A \subseteq B$，则 $A \in \mathcal{I}$
    - 由于 $\mathcal{I}$ 满足此性质，所以称其为遗传的
    - $\varnothing \in \mathcal{I}$
  3. 若 $A \in \mathcal{I}, B \in \mathcal{I}$ 且 $|A| \lt |B|$，则 $\exists\  x \in B - A$，使得 $A \cup \{x\} \in \mathcal{I}$
    - 此性质说明 $M$ 满足交换性质

###### 图拟阵

- 图拟阵 $M_{G}=\left(S_{G}, \mathcal{I}_{G}\right)$ 定义在一个给定的无向图 $G=(V, E)$ 上
  - $S_G$ 定义为 $E$ 即 $G$ 的边集
  - 如果 $A \subseteq E$， 则 $A\in \mathcal{I}$ 当且仅当 $A$ 是无圈的
  - 一组边 $A$ 是独立的，当且仅当子图 $G_A = (V, A)$ 形成一个森林
    - 可能不连通的无向无环图称为森林
    - 边集中包含所有的顶点 $V$
  

###### 定理 16.5
- 如果 $G=(V, E)$ 是一个无向图，则 $M_G = (S_G, \mathcal{I}_G)$ 是一个拟阵

- 证明：
  1. $S_G = E$，因此 $S_G$ 是一个有限集
  2. $\mathcal{I}_G$ 是遗传的，因为森林的子集还是森林
    - 即从无圈的边集中删除边不会产生圈
  3. 假定 $G_A=(V, A)$ 和 $G_B=(V, B)$ 是 $G$ 的森林，且 $|B| \ge |A|$
    - 结论： $F=(V_F, E_F)$ 恰好包含 $|V_F| - |E_F|$ 棵树
      - 连通的无向无环图称为自由树，简称树
      - 假定 $F$ 包含 $t$ 棵树，其中第 $i$ 棵树包含 $v_i$ 个顶点和 $e_i$ 条边，则有：
        - $\begin{aligned}\left|E_{F}\right| &=\sum_{i=1}^{t} e_{i} \\ &=\sum_{i=1}^{t}\left(v_{i}-1\right) \quad \text { (by Theorem B.2) } \\ &=\sum_{i=1}^{t} v_{i}-t \\ &=\left|V_{F}\right|-t \end{aligned}$
    - 森林中 $G_A$ 中包含 $|V| - |A|$ 棵树， $G_B$ 中包含 $|V| - |B|棵树$， 即 $G_B$ 中树的数目比 $G_A$ 少
      - 由此可得 $G_B$ 中必然包括某棵树 $T$ ， $T$ 不在森林中 $G_A$ 中
      - 由于 $T$ 是连通的，则 $T$ 中必然包含一条边 $(u, v)$， 其在森林 $G_A$ 中属于两棵不同的树，即 $(u, v) \in B-A$
      - 可以将 $(u, v)$ 加入 $G_A$ 中，而不会产生圈， 即 $(u, v) \cup A \in \mathcal{I}_G$
    - 综上，可得 $M_G$ 满足交换性质
 - 结合 1，2，3，可证明 $M_G$ 是拟阵

##### 其它定义

- 给定拟阵 $M=(S, \mathcal{I})$
  - $A \in \mathcal{I} $ 且 $x \notin A$，如果 $A \cup x \in \mathcal{I}$， 则 $x$ 是 $A$ 的一个扩展
  - 如果 $M$ 中的一个独立子集 $A$ 不存在扩展，则称它是最大的 

###### 定理 16.6 拟阵中的所有最大独立子集都具有相同的大小

- 证明可通过反证法结合拟阵的交换性质进行

### 加权拟阵

- 对于拟阵 $M = (S, \mathcal{I})$， 为每个 $x \in S$ 赋予一个严格大于 $0$ 的权重 $w(x)$， 则称 $M$ 为加权的
- 通过求和，可将权重函数 $w$ 扩展到 $S$ 的任意子集 $A$
  - $$w(A)=\sum_{x \in A} w(x)$$

#### 加权拟阵上的贪心算法

- 很多可用贪心算法求解的最优化问题都可形式化为在一个加权拟阵中寻找最大权重独立子集的问题
  - 这种最大权重的独立子集称为拟阵的最优子集
  - 由于 $w(x) > 0$， 则最优子集必然是最大的独立子集 

##### 最小生成树问题

- 问题描述
  - 给定一个连通无向图 $G=(V, E)$ 和一个长度函数 $w$，使得 $w_e$ 表示边 $e$ 的长度（正值）， 目的是找到一个边的子集，能连接所有的顶点，且具有最小总长度
- 转换为最优子集问题
  - 考虑加权拟阵 $M_G$， 其权重函数 $w^{\prime}(e) = w_0 - w(e)$， $w_0$ 为大于最大边的值
  - 每个最大独立子集 $A$， 均构成一棵连通，无向的生成树，有 $|V| - 1$ 条边， 对所有的 $A$, 有：
    - $w^{\prime}(A)=\sum_{e \in A} w^{\prime}(e)=\sum_{e \in A}\left(w_{0}-w(e)\right)=(|V|-1) w_{0}-\sum_{e \in A} w(e)=(|V|-1) w_{0}-w(A)$
    - $w(A)$ 的最小值对应 $w^{\prime}(A)$ 的最大值，因此最小生成树问题即可等价于求解拟阵中的最优子集 $A$

##### 求解最优子集 $A$ 的算法

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200414131759.png width=800>

###### 引理 16.7 拟阵具有贪心选择的性质
- 假定 $M = (S, \mathcal{I})$ 是一个加权拟阵，加权函数为 $w$，且 $S$ 已按权重单调递减的顺序排序
- 令 $x$ 是 $S$ 中第一个满足 ${x}$ 独立的元素（如果存在）。 如果存在这样的 $x$，那么存在 $S$ 的一个最优子集 $A$ 包含 $x$

- 证明可利用拟阵的交换性质

###### 引理 16.8 
- 令 $M=(S, \mathcal{I})$ 是一个拟阵。如果 $x$ 是 $S$ 中的一个元素，而且是 $S$ 的某个独立子集 $A$ 的一个扩展，则 $x$ 也是 $\varnothing$ 的一个扩展

- 证明可借助拟阵的遗传特性实现

###### 推论 16.9
- 令 $M=(S, \mathcal{I})$ 是一个拟阵。如果 $x$ 是 $S$ 中的一个元素， 且它不是 $\varnothing$ 的一个扩展，那么它也不是 $S$ 的任何独立子集 $A$ 的扩展

- 引理 16.8 的逆否命题

###### 引理 16.10 拟阵具有最优子结构性质
- 令 $M=(S, \mathcal{I})$ 是一个加权拟阵， $x$ 是 $S$ 中第一个被 $GREEDY$ 算法选出的元素
- 则接下来寻找一个包含 $x$ 的大权重独立子集问题归结为寻找加权拟阵 $M^{\prime} = (S^{\prime}, \mathcal{I}^{\prime})$ 的一个最大权重独立子集问题，其中
  - $S^{\prime}=\left\{y \in S : \{x, y\} \in \mathcal{I}\right\}$
  - $\mathcal{I}^{\prime}=\{B \subseteq S-\{x\} : B \cup\{x\} \in \mathcal{I}\}$
  - $M^{\prime}$ 的权重函数即为 $M$ 的权重函数，但只限于 $S^{\prime}$ 中的元素

- 证明： 
  1. 若 $A$ 是 $M$ 的一个包含 $x$ 的最大权重的独立子集，则 $A^{\prime} = A - \{x\}$ 是 $M^{\prime}$ 的一个独立子集
  2. 任何 $M^{\prime}$ 的独立子集 $A^{\prime}$ 可生成 $M$ 的独立子集 $A = A^{\prime} \cup \{x\}$
  3. 对于 1， 2 , 均有 $w(A) = w(A^{\prime}) + w(x)$
  - 结合1，2，3 可得 $M$ 包含 $x$ 的最大权重独立子集必然生成 $M^{\prime}$ 的最大权重独立子集
    - 反之亦然

###### 定理 16.11 拟阵上贪心算法的正确性

- 证明：
  1. 由推论 16.9， 算法跳过的任何不是 $\varnothing$ 的扩展的起始元素可永远丢弃，
    - 因为这些元素永远也不会再被用到
  2. 引理 16.7 表明一旦算法选出第一个 $x$， 则将 $x$ 加入 $A$ 中不会导致错误的结果
    - 因为必然存在包含 $x$ 的最优子集
  3. 引理 16.10 说明在找出一个元素 $x$ 后，剩下的问题就是寻找拟阵 $M^{\prime}$ 的最优子集了
  - 由1，2，3 可证得拟阵上贪心算法的确性

### 练习

#### 16.4-1
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200423091735.png width=800>

- 证明：
  - 由题设可知， $S$ 是有限集
  - 设 $B \in \mathcal{I}$， 如果 $A \subseteq B$， 则 $|A| \le |B| \le k$，所以可得 $A \in \mathcal{I}$，满足遗传特性
  - 如果 $A, B \in \mathcal{I}$ 且 $|A| < |B|$， 则对于任意的  $x \in B - A$， $|A \cup \{x\}| \le k$， 即 $A \cup \{x\} \in  \mathcal{I}$，满足交换特性
  - 由此可得 $(S, \mathcal{I_k})$ 是一个拟阵 

##### 16.4-2
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200423092144.png width=800>


- 证明
  - $S$ 是非空的
  - 如果 $B\subseteq A$，由于 $A$ 中的列线性无关，所以 $B$ 中的列也线性无关，即 $B \in \mathcal{I}$，满足遗传特性
  - 如果 $A, B \in \mathcal{I}$ 且 $|A| < |B|$, 如果对于任意的 $x$, $A \cup \{x\} \notin \mathcal{I}$， 说明 $A$ 中的每个元素都可表示为 $B$ 中元素的线性组合，这与 $|A| < |B|$ 矛盾，假设不成立。 由此可证明交换特性。
  - 综上可证得 $(S, \mathcal{I})$ 是一个拟阵 

#### 16.4-3
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200423092227.png width=600>

- 证明
  - $S$ 是非空的
  - 设 $A^{\prime} \in \mathcal{I^{\prime}}$， 则 $\mathcal{I}$ 中存在一个最大的独立子集 $A$, 使得 $A \subseteq S - A^{\prime}$。如果 $B^{\prime} \subseteq A^{\prime}$， 则 $A \subseteq  S - A^{\prime}\subseteq S - B^{\prime}$, 说明 $B \in \mathcal{I^{\prime}}$，满足遗传特性
  - 下面证明交换性
    - 设 $A^{\prime}, B^{\prime} \in \mathcal{I^{\prime}}$ 且 $|B^{\prime}| < |A^{\prime}|$
    - 则 $\mathcal{I}$ 中存在最大独立子集 $A, B$， 使得 $A\subseteq S-A^{\prime}, B \subseteq S - B^{\prime}$
    - 设 $x \in A^{\prime} - B^{\prime}$， 则 $x \in A^{\prime}, x \notin A$
      - 如果存在 $x \notin B$， 则 $B \subseteq S - (B^{\prime} \cup x)$， 说明 $B^{\prime} \cup {x} \in \mathcal{I^{\prime}}$
      - 如果对于所有的$x$， $x \in B$， 则令 $C = B - x$
        - 由于 $C \subseteq B$， 由拟阵的遗传特性可得 $C \in \mathcal{I}$
        - 由于 $|C| < |A|$，所以由拟阵的交换特性可得，存在 $y \in A - C$， 使得 $C \cup \{y\} \in \mathcal{I}$
          - 由于 $y \in A - C = A - (B - \{x\})$, 又因为 $x \notin A$， 可得 $y \notin B$，且 $y \neq x$
          - 可得 $(C \cup \{y\}) \subseteq S - (B \cup {x})$， 由于拟阵的最大独立子集的大小相同，所以 $C \cup {y}$ 也是 $（S, \mathcal{I}）$ 的一个最大独立子集，由此可得 $B \cup \{x\} \in \mathcal{I^{\prime}}$
    - 综上可证明交换性
  - 综上可证得 $(S, \mathcal{I^{\prime}})$ 是一个拟阵

#### 16.4-4
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200423094734.png width=800>


- 证明
  - $S$ 是非空的
  - 如果 $A \in \mathcal{I}$ 且 $B \subseteq A$， 则 $B$ 也满足与划分中所有子集都最多只有一个共同元素的条件，即 $B \in \mathcal{I}$ 满足交换特性
  - 如果 $A, B \in \mathcal{I}$ 且  $|B| > |A|$， 则存在一个 $j$， 使得 $|B \cap S_j| = 1$， 但$|A \cap S_j| = 0$
    - 取 $a_j = B \cap S_j$， 则 $a \in B - A$
    - 由于 $S_1, S_2, \cdots, S_k$ 中的元素各不相同，可得：
      - $|A \cup \{a_j\} \cap S_i| =\left \{ \begin{aligned} 
      &1 && if \ i == j \\
      & 0 && if \ i \neq j
      \end{aligned} \right. $
    - 由此可得 $A\cup \{a_j\} \in \mathcal{I}$，可证明交换性
  - 综上可得 $(S, \mathcal{I})$ 是一个拟阵

#### 16.4-5
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200423102136.png width=800>

- 设权重的最大值为 $x$， 则任取一个正数 $y$， 使得 $y > x$， 然后用 $y$ 减去所有的权重得到新的权重。如此，即可将最小权重最大独立子集问题转换为标准的加权拟阵问题

## 16.5 用拟阵求解任务调度问题

##### 问题描述

- 单位时间任务是严格按照一个时间单位来完成的作业
- 给定一个单位时间任务的有限集合 $S$， 对 $S$ 的一个调度是指 $S$ 的一个排列，指明了任务执行的顺序
  - 第一个被调度的任务起始时间为 $0$，终止时刻为 $1$，第二个任务开始于时刻 1， 结束于时刻 2，以此类推
***
- 单处理器上带截止时间和惩罚单位的单位时间任务调度问题有如下输入
  - $n$ 个单位时间的任务集合 $S = \{a_1, a_2, \cdots, a_n\}$
  - $n$ 个整数截止时间 $d_1, d_2, \cdots, d_n$
    - $1 \le d_i \le n$
  - $n$ 个非负权重（惩罚） $w_1, w_2, \cdots, w_n$， 若任务 $a_i$ 在时间 $d_i$ 之间没有完成，就会受到 $w_i$ 的惩罚，如果在 $d_i$ 之前完成，就不会受到惩罚
- 目标是找到 $S$ 的一个调度方案，能最小化超时导致的惩罚总和


##### 算法分析

- 对于任意调度方案，可以将其转换为提前优先形式，即将提前的任务都置于延迟的任务之前
  - 如是某个提前的任务 $a_i$ 位于延迟的任务 $a_j$ 之后，则可以交换 $a_i$ 与 $a_j$ 的位置，如此 $a_i$ 仍是提前的， $a_j$ 仍是延迟的
- 在转换为提前优先形式的基础上，可进一步将其转换为规范形式，即提前任务按截止时间单调递增的顺序排列
  - 如果 $a_i$ 和 $a_j$ 分别在 $k, k+1$ 时刻完成，但 $d_j<d_i$，如此便可交换 $a_i$ 与 $a_j$ 的位置
    - 交换的， $a_j$ 必仍是提前的
    - 由于 $k+1 \le d_j \lt d_i$, 此时 $a_i$ 也仍是提前的
***
- 如此，寻找最优调度方案的问题就可转换为寻找提前任务子集 $A$ 的问题
- 如果 $A$ 中存在一个调度方案，使得其中所有的任务均不延迟，则称 $A$ 是独立的
- 一个调度方案的提前任务集合构成了一个独立任务集，记为 $\mathcal{I}$ 

###### 引理 16.12
- 对于任意任务集合 $A$， 下面性质是等价的：
  1. $A$ 是独立的
  2. 对于 $t=0, 1, 2, \cdots, n$，有 $N_t(A) \le t$
    - $N_t(A)$ 表示 $A$ 中截止时间小于等于 $t$ 的任务数， $N_0(A) = 0$
  3. 如果 $A$ 中的任务按截止时间单调递增的顺序调度，则不会有任务延迟


###### 定理 16.13
- 如果 $S$ 是一个给定了截止时间的单位时间任务集合， $\mathcal{I}$ 是所有独立任务集合的集合，则对应的系统 $(S, \mathcal{I})$ 是一个拟阵


- 证明
  1. 每个独立任务的子集必然也是独立的，满足遗传特性
  2. 证明交换性
    - 设 $A, B$ 为独立的任务集合， 且 $|B| > |A|$
      - 令 $k$ 是满足 $N_t(B) \le N_t(A)$ 的最大的 $t$
        - $t$ 肯定是存在的，因为 $N_0(A) = N_0(B) = 0$
        - 由于 $|B| > |A|$，所以 $k < n$
      - 当 $k+1 \le j \le n$ 时， 有 $N_j(B) > N_j(A)$，则说明 $B$ 中截止时间小于等于 $k+1$ 的任务数多于 $A$
      - 则存在 $a_i \in B - A$，且 $d_i \le k+1$， 令 $A^{\prime} = A \cup \{a_i\}$
      - 可借助引理 16.12 中的性质 2 来证明 $A^{\prime}$ 是独立的
        - 当$0 \le t \le k$时， 有 $N_t(A^{\prime}) = N_t(A) \le t$
        - 当 $k + 1\le t \le n$ 时， 有 $N_t(A^{\prime}) \le N_t(B) \le t$
        - 由此可得 $A^{\prime}$ 是独立的
  - 综合1，2 可证得 $(S, \mathcal{I})$ 是一个拟阵， 可以采用加权拟阵上的贪心算法进行求解

##### 实例

- 任务实例如下

| $a_i$ | 1  | 2  | 3  | 4  | 5  | 6  | 7  |
|-------|----|----|----|----|----|----|----|
| $d_i$ | 4  | 2  | 4  | 3  | 1  | 4  | 6  |
| $w_i$ | 70 | 60 | 50 | 40 | 30 | 20 | 10 |


- 贪心算法按顺序选择 $a_1, a_2, a_3, a_4$， 然后拒绝 $a_5$ （因为 $N_4({a_1, a_2, a_3, a_4, a_5}) = 5$） 和 $a_6$（因为 $N_4({a_1, a_2, a_3, a_4, a_6}) = 5$），最后接受 $a_7$，最终的最优调度为:
  - $$<a_2, a_4, a_1, a_3, a_7, a_5, a_6>$$
- 总惩罚为 $w_5 + w_6 = 50$

##### 代码实现

In [0]:
%%writefile ch16/task_scheduling.py
from collections import namedtuple
import bisect
from operator import attrgetter


class Task:
  def __init__(self, d, w):
    self.d = d
    self.w = w

  def __repr__(self):
    class_name = type(self).__name__
    return "{}({!r}, {!r})".format(class_name, self.d, self.w)


def task_scheduling(tasks):
  """单任务时间调度，假设任务已按惩罚时间单调递减的顺序排序"""

  N = [0] * len(tasks)  #　储存 N_t 的值
  tasks = sorted(tasks, key=attrgetter('w'), reverse=True)
  early_tasks, late_tasks = [], []
  w = 0
  for task in tasks:
    tmp = N[:] # 复制 N，防止延迟任务更新 N
    i = task.d - 1
    while i < len(tasks):
      tmp[i] += 1
      if tmp[i] > i + 1:
        late_tasks.append(task)
        w += task.w
        break
      i += 1
    if i == len(tasks):
      early_tasks.append(task)
      N = tmp
  
  return sorted(early_tasks, key=attrgetter('d')) + late_tasks, w


if __name__ == "__main__":
  tasks = []
  for d, w in zip([4, 2, 4, 3, 1, 4, 6], [70, 60, 50, 40, 30, 20, 10]):
   tasks.append(Task(d, w))
  print("任务为： \n", "\n".join(repr(task) for task in tasks), sep='')
  res, w = task_scheduling(tasks)
  print("最终的任务调度序列为: \n", "\n".join(repr(task) for task in res), sep='')
  print("总的超时惩罚为: ", w)

Overwriting ch16/task_scheduling.py


In [0]:
!python ch16/task_scheduling.py

任务为： 
Task(4, 70)
Task(2, 60)
Task(4, 50)
Task(3, 40)
Task(1, 30)
Task(4, 20)
Task(6, 10)
最终的任务调度序列为: 
Task(2, 60)
Task(3, 40)
Task(4, 70)
Task(4, 50)
Task(6, 10)
Task(1, 30)
Task(4, 20)
总的超时惩罚为:  50


### 练习题

#### 16.5-1
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200424083802.png width=800>

In [0]:
import ch16.task_scheduling
imp.reload(ch16.task_scheduling)
from ch16.task_scheduling import task_scheduling

In [0]:
tasks = []
for d, w in zip([4, 2, 4, 3, 1, 4, 6], [70, 60, 50, 40, 30, 20, 10]):
  tasks.append(Task(d, 80-w))
print("任务为： \n", "\n".join(repr(task) for task in tasks), sep='')
res, w = task_scheduling(tasks)
print("最终的任务调度序列为: \n", "\n".join(repr(task) for task in res), sep='')
print("总的超时惩罚为: ", w)

任务为： 
Task(4, 10)
Task(2, 20)
Task(4, 30)
Task(3, 40)
Task(1, 50)
Task(4, 60)
Task(6, 70)
最终的任务调度序列为: 
Task(1, 50)
Task(3, 40)
Task(4, 60)
Task(4, 30)
Task(6, 70)
Task(2, 20)
Task(4, 10)
总的超时惩罚为:  30


### 16.5-2
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200424083835.png width=800>

- 与计数排序类似
- 设任务集合的中最大的截止完成时间为 $n$， 则创建一个长度为 $n$ 的数组 $N_t$，初始全部置0
- 遍历 $A$ 中的每个元素 $a$， $N_t[a.d] = N_t[a.d] + 1$
- 然后遍历数组 $N_t$, 进行下述计算 $N_t[i] = N_t[i] + N_t[i-1]$
- 遍历数组 $N_t$， 如果出现 $N_t[i] > i$， 则说明 $A$ 不独立
  - 如果完全符合要求，则说明 $A$ 是独立的

## 思考题

### 16-1 找零问题
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200424100814.png width=800>

#### a

- 尽可能的用大面额的硬币进行找零，如果剩余的钱数少于最大面额的硬币，则用下一个更小面额的硬币进行找零，直至找零完成
- 证明：
  1. 若$n$是正整数，则用$25$美分、$10$美分、$5$美分和$1$美分等尽可能少的硬币找出的$n$美分零钱中，至多有$2$个$10$美分、至多有$1$个$5$美分、至多有$4$个$1$美分硬币，而不能有$2$个$10$美分和$1$个$5$美分硬币
  - 1 中的结论具体可以通过反证法证明
  - 为了满足1， ，必须应先选择 25 美分找零，当钱数小于 $25$ 美分时，开始用 10 美分找零，以此类推

#### b

- 证明：
  1. 设找零时， 面额为 $c^i$ 的硬币需要 $x_i$ 个， 则当 $i \lt k$ 时，$x_i \lt c$
    - 因为一旦 $x_i > c$, 便可用一张面额为 $c^{i+1}$ 和 $x_i-c$ 张面额为 $c^i$ 的硬币进行替换，从而得到最优的解
  - 设总金额为 $V$， 为了满足 1 中的要求， $x_i$ 需按下式求出：
    - $x_i = \left\{ \begin{aligned}
    &\lfloor {V}/{c^k}\rfloor  && if \ i = k \\
    &\lfloor ({V}\mod {c^{i+1}}) / c^i \rfloor && if \ i \lt k 
    \end{aligned}\right.$
    - 贪心算法可以保证结果按上式求出

#### c

- 设硬币面额为 $\{1, 3, 4 \}$， 则当总金额为 $6$ 时，按贪心算法，结果为 $\{4, 1, 1\}$， 而最佳的结是为 $\{3, 3\}$

#### d

- [参考资料](http://www.ccs.neu.edu/home/jaa/CSG713.04F/Information/Handouts/dyn_prog.pdf)

##### 算法分析

- 设硬币的面额为 $d_1, d_2, \cdots, d_k$
- 设 $c[i]$ 为总金额为 $i$ 时，找零的总硬币数目， 则 $c[i]$ 可按下式求出：
  - $c[i] = \left \{ \begin{aligned}
    &0 && if \ i = 0 \\
    &\min_\limits{j: d_j \le i} \{1 + c[i-d_j]\} && if \ i > 0
   \end{aligned} \right.$
- 为了重构最优解，需要用 $b[i]$ 来记录 $c[i]$ 取得最小值时，所选取的硬币 $j$

##### 代码实现

In [0]:
%%writefile ch16/coin_change.py
"""16-1 找零问题"""

def coin_change(d, v):
  """找零问题，需要确保 d 按升序排列"""
  c = [0] * (v + 1)
  b = [0] * (v + 1)
  for i in range(1, v+1):
    c[i] = float('inf')
    for j in range(len(d)):
      if d[j] > i:
        break
      t = 1 + c[i-d[j]]
      if t < c[i]:
        c[i], b[i] = t, j
  construct_solution(d, v, b)


def construct_solution(d, v, b):
  res = [0] * len(d)  # 每种硬币所需要的数量
  while v > 0:
    res[b[v]] += 1
    v -= d[b[v]]
  print("找零选择为：")
  sums = 0
  for i, nums in enumerate(res):
    sums += nums
    if nums > 0:
      print("\t{} 个 {} 美分的硬币".format(nums, d[i]))
  print("总的硬币数量为： ", sums)


if __name__ == "__main__":
  import random
  d = sorted(random.sample(range(100), 10))
  v = random.randint(100, 200)
  print("硬币的面值为： ", d)
  print("总金额为：", v, '美分')
  coin_change(d, v)

Overwriting ch16/coin_change.py


In [0]:
!python ch16/coin_change.py

硬币的面值为：  [3, 4, 12, 27, 49, 54, 56, 67, 86, 94]
总金额为： 161 美分
找零选择为：
	1 个 67 美分的硬币
	1 个 94 美分的硬币
总的硬币数量为：  2


##### 复杂度分析

- 算法共执行了 $nk$ 次循环，每次循环用时为 $O(1)$，总的时间复杂度为 $O(nk)$

### 16-2 最小平均完成时间调度问题
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200424101041.png width=800>

#### a

- 依次选择 $p_i$ 最小的活动， 即可得到最优解
- 首先易得问题有最优子结构，只需证明贪心选择能得到局部最优解即可
  - 设 $a_1$ 是持续时间最小的活动，假设存在一个最优解 $O$，其第一个活动不是 $a_1$, 设其为 $a_x$
  - 由假设可得 $p_1 \le p_x$，则通过交换 $a_1, a_x$， 可以让 $a_x$ 到 $a_1$ 的活动结束时间均减小 $p_x - p_1$， 从而得到更优的解
  - 由此可得假设不成立， $O$ 中的第一个活动一定为 $a_1$
- 只需将活动按照持续时间排序，即可得得出活动序列。时间复杂度为 $O(nlgn)$


#### b

- 基本思路仍然是让活动尽可能早的结束，如此即可得到最优解。证明方法与 a 中基本相同
- 具体实现可以借助最小优先队列，最小优先队列的比较的属性是任务的剩余处理时间。
  - 可以按照任务释放时间的顺序添加任务，在两个任务的释放间隔，按照最小任务剩余时间处理任务。如果在间隔内，能处理完任务，则将任务从最小优先队列中剔除。然后接着处理下一个间隔时间最短的任务，直至到达下一个任务的释放时间。 如果到了下一个任务的释放时间，正在处理的任务没有完成，则将其运行时间减小， 但仍留在队列中。然后加入新的任务后，再继续上述操作
  - 当所有的任务添加完毕后，即可按照最小优先队列的顺序依次处理队列中剩余的任务
- 对 $n$ 个任务按任务释放时间进行排序操作，需要 $O(n\ lgn)$。最小优先队列的处理也需要 $O(n\lg n)$，因此整体的时间复杂度为 $O(n \lg n)$

##### 代码实现

- 需补全

### 16-3 无环子图
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200424101126.png width=800>

- 与图相关，看完图的相关章节后再补

### 16-4 调度变形问题

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200424101214.png width=800>

#### a

- 如果存在不晚于 $a_j$ 的截止时间 $d_j$ 的空时间槽，说明 $a_j$ 在已经处理的任务中，能哆提前完成，说明其在提前完成任务集合 $A$ 中。将其分配到其中最晚的时间槽，是为了让$A$中的任务按结束时间单调递增排列，即满足引理 16.12 中的 3。 从而保证 $A$ 是最大的独立子集
- 如果不存在这样的时间槽，说明 $a_j$ 不在提前任务集 $A$ 中，则为了取得最优解，其必将延后完成。将其分配到最晚的空时间槽，不会影响到后续活动是否在 $A$ 中的判断
- 基本思路与加权拟阵基本相同，即尽可能的让延迟惩罚大的任务提前结束

##### b

- 看完 21.3 节后再回来补

### 16-5 离线缓存问题
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200424101506.png width=800>

- [参考资料](http://web.cs.wpi.edu/~cs2223/b13/HW/HW5/Solutions/solutions_cs2223_B13_HW5.pdf)
- 具体待补全